# with 语句和上下文管理器

```python
# create/aquire some resource
...
try:
    # do something with the resource
    ...
finally:
    # destroy/release the resource
    ...
```

处理文件，线程，数据库，网络编程等等资源的时候，我们经常需要使用上面这样的代码形式，以确保资源的正常使用和释放。

好在`Python` 提供了 `with` 语句帮我们自动进行这样的处理，例如之前在打开文件时我们使用： 

In [1]:
with open('my_file', 'w') as fp:
    # do stuff with fp
    data = fp.write("Hello world")

这等效于下面的代码，但是要更简便：

In [2]:
fp = open('my_file', 'w')
try:
    # do stuff with f
    data = fp.write("Hello world")
finally:
    fp.close()

## 上下文管理器

其基本用法如下：
```
with <expression>:
    <block>
```

`<expression>` 执行的结果应当返回一个实现了上下文管理器的对象，即实现这样两个方法，`__enter__` 和 `__exit__`：

In [3]:
print( fp.__enter__)
print (fp.__exit__)

<built-in method __enter__ of _io.TextIOWrapper object at 0x000001B348482668>
<built-in method __exit__ of _io.TextIOWrapper object at 0x000001B348482668>


`__enter__` 方法在 `<block>` 执行前执行，而 `__exit__` 在 `<block>` 执行结束后执行：

比如可以这样定义一个简单的上下文管理器：

In [4]:
class ContextManager(object):
    
    def __enter__(self):
        print ("Entering")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print ("Exiting")

使用 `with` 语句执行：

In [5]:
with ContextManager():
    print( "  Inside the with statement")

Entering
  Inside the with statement
Exiting


即使 `<block>` 中执行的内容出错，`__exit__` 也会被执行：

In [6]:
with ContextManager():
    print (1/0)

Entering
Exiting


ZeroDivisionError: division by zero

## `__`enter`__` 的返回值

如果在 `__enter__` 方法下添加了返回值，那么我们可以使用 `as` 把这个返回值传给某个参数：

In [7]:
class ContextManager(object):
    
    def __enter__(self):
        print ("Entering")
        return "my value"
    
    def __exit__(self, exc_type, exc_value, traceback):
        print ("Exiting")

将 `__enter__` 返回的值传给 `value` 变量：

In [8]:
with ContextManager() as value:
    print (value)

Entering
my value
Exiting


一个通常的做法是将 `__enter__` 的返回值设为这个上下文管理器对象本身，文件对象就是这样做的：

In [9]:
fp = open('my_file', 'r')
print (fp.__enter__())
fp.close()

<_io.TextIOWrapper name='my_file' mode='r' encoding='cp936'>


In [10]:
import os
os.remove('my_file')

实现方法非常简单：

In [13]:
# 没看懂这是要干什么，意义何在
class ContextManager(object):
    
    def __enter__(self):
        print ("Entering")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        print ("Exiting")

In [14]:
with ContextManager() as value:
    print( value)

Entering
Exiting


## 错误处理

上下文管理器对象将错误处理交给 `__exit__` 进行，可以将错误类型，错误值和 `traceback` 等内容作为参数传递给 `__exit__` 函数：

In [15]:
class ContextManager(object):
    
    def __enter__(self):
        print ("Entering")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print ("Exiting")
        if exc_type is not None:
            print ("  Exception:", exc_value)

如果没有错误，这些值都将是 `None`, 当有错误发生的时候：

In [16]:
with ContextManager():
    print (1/0)

Entering
Exiting
  Exception: division by zero


ZeroDivisionError: division by zero

在这个例子中，我们只是简单的显示了错误的值，并没有对错误进行处理，所以错误被向上抛出了，如果不想让错误抛出，只需要将 `__exit__` 的返回值设为 `True`： 

In [17]:
class ContextManager(object):
    
    def __enter__(self):
        print( "Entering")
    
    def __exit__(self, exc_type, exc_value, traceback):
        print ("Exiting")
        if exc_type is not None:
            print (" Exception suppresed:", exc_value)
            return True

In [18]:
with ContextManager():
    print (1/0)

Entering
Exiting
 Exception suppresed: division by zero


在这种情况下，错误就不会被向上抛出。

## contextlib 模块

很多的上下文管理器有很多相似的地方，为了防止写入很多重复的模式，可以使用 `contextlib` 模块来进行处理。

最简单的处理方式是使用 `closing` 函数确保对象的 `close()` 方法始终被调用：

In [22]:
from contextlib import closing
import urllib

with closing(urllib.request.urlopen('http://www.baidu.com')) as url:
    html = url.read()

print (html[:100])

b'<!DOCTYPE html><!--STATUS OK-->\n\n\n    <html><head><meta http-equiv="Content-Type" content="text/html'


另一个有用的方法是使用修饰符 `@contextlib`：

In [23]:
from contextlib import contextmanager

@contextmanager
def my_contextmanager():
    print ("Enter")
    yield
    print ("Exit")

with my_contextmanager():
    print ("  Inside the with statement")

Enter
  Inside the with statement
Exit


`yield` 之前的部分可以看成是 `__enter__` 的部分，`yield` 的值可以看成是 `__enter__` 返回的值，`yield` 之后的部分可以看成是 `__exit__` 的部分。

使用 `yield` 的值：

In [25]:
@contextmanager
def my_contextmanager():
    print ("Enter")
    yield "my value"
    print ("Exit")
    
with my_contextmanager() as value:
    print (value)

Enter
my value
Exit


错误处理可以用 `try` 块来完成：

In [26]:
@contextmanager
def my_contextmanager():
    print ("Enter")
    try:
        yield
    except Exception as exc:
        print ("   Error:", exc)
    finally:
        print ("Exit")

In [27]:
with my_contextmanager():
    print( 1/0)

Enter
   Error: division by zero
Exit


对于之前的数据库 `transaction` 我们可以这样定义：

In [28]:
@contextmanager
def transaction(connection):
    cursor = connection.cursor()
    try:
        yield cursor
    except:
        connection.rollback()
        raise
    else:
        connection.commit()